# AutoGEN 第五课： 如何实现组聊天

上一节中，我们提到多个agent的对话模式，其实你仔细想一下就可以发现，这里虽然提到了多个agent，但是同一时间只有2个agent在聊天。
倘若如此，凭什么称之为目前最强的agent框架呢？

在这一节中，我们就来讲讲这个里面最核心的一个功能 -- 组聊天。 真正意义上的多个agent聊天。

1. 首先我们构建了3个agent， 
    - user_agent: 作为人类的代表
    - coder: 搬砖的
    - pm: 产品经理
2. 构建一个groupchat, 用来把这些agent装在一起
3. 构建一个group chat manager, 用来管理group chat. 注意，这个manager本质上也是一个agent！它有这么2个重要的作用。
    - 拿到消息后，将消息广播给组内的所有agent，也就是说，组内的所有agent所获得的信息都是一样的，共享的。
    - 选择当前回合最合适的agent来发表意见，当这个agent发表完以后，根据内容再选择下一个agent来发表意见。




## 1. 配置环境

1. 你需要安装autogent的库， 要求python版本>=3.8
2. 配置openai 或者 azure openai的api key, 并将key写入到一个名为 "OAI_CONFIG_LIST"的文件中，并将该文件放在项目目录下。或者将其配置到你的环境变量中。

OAI_CONFIG_LIST的内容结构如下：
```python
[
    {
        'model': 'gpt-4',
        'api_key': '<your OpenAI API key here>',
    },
    {
        'model': 'gpt-4',
        'api_key': '<your Azure OpenAI API key here>',
        'api_base': '<your Azure OpenAI API base here>',
        'api_type': 'azure',
        'api_version': '2023-06-01-preview',
    },
    {
        'model': 'gpt-4-32k',
        'api_key': '<your Azure OpenAI API key here>',
        'api_base': '<your Azure OpenAI API base here>',
        'api_type': 'azure',
        'api_version': '2023-06-01-preview',
    },
]

In [1]:
%pip install pyautogen~=0.1.0 -q

Note: you may need to restart the kernel to use updated packages.


## 2.构建group chat 

In [7]:
import autogen

config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-4", "gpt-4-0314", "gpt4", "gpt-4-32k", "gpt-4-32k-0314", "gpt-4-32k-v0314"],
    },
)

llm_config = {"config_list": config_list}
user_proxy = autogen.UserProxyAgent(
   name="User_proxy",
   system_message="A human admin.",
   code_execution_config={"last_n_messages": 2, 
                          "work_dir": "coding",
                          "use_docker": False},
   human_input_mode="TERMINATE"
)
coder = autogen.AssistantAgent(
    name="Coder",
    llm_config=llm_config,
)
pm = autogen.AssistantAgent(
    name="Product_manager",
    system_message="Creative in software product ideas.",
    llm_config=llm_config,
)
groupchat = autogen.GroupChat(agents=[user_proxy, coder, pm], messages=[], max_round=12)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

## 3.发起会话

In [8]:
user_proxy.initiate_chat(manager,
                         message="在arxiv上找到关于gpt4的最新论文，并且说明这个论文的研究成果可以应用到哪些领域。")


User_proxy (to chat_manager):

在arxiv上找到关于gpt4的最新论文，并且说明这个论文的研究成果可以应用到哪些领域。

--------------------------------------------------------------------------------
Coder (to chat_manager):

要在arXiv上找到关于gpt4的最新论文，我们将使用Python的arxiv包。arxiv是一个用于检索arXiv预印本的Python库。以下是我们需要执行的步骤：

1. 首先，我们需要确保我们安装了arxiv库。我们可以使用Python的包管理器pip来完成这个工作。
    
    执行以下代码安装arxiv库：

```bash
pip install arxiv
```
  
2. 使用arxiv库在arXiv上搜索关于gpt-4的最新论文。

执行以下代码搜索gpt4的最新论文：

```python
# filename: arxiv_search.py

import arxiv

# Search for the latest papers on gpt-4
search = arxiv.Search(
  query = "gpt-4",
  max_results = 1,
  sort_by = arxiv.SortCriterion.SubmittedDate
)

for result in search.get():
  print(f"Title: {result.title}")
  print(f"Authors: {result.authors}")
  print(f"Abstract: {result.summary}")
  print(f"Url: {result.pdf_url}")
```

以上代码会找到最近一篇关于gpt-4的文章，并打印出相关信息如标题，作者，摘要和论文的url链接。之后我们再按您的要求，根据摘要里的内容分析这个论文的研究成果可以应用到哪些领域。

--------------------------------------------------------------------------------

>>>>>>>> U

Timeout: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=60)